# Урок 2. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes).

In [152]:
import pandas as pd

In [153]:
df = pd.read_csv('train.csv')
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [154]:
df.info()
df[['Age', 'Cabin', 'Embarked']].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Age         177
Cabin       687
Embarked      2
dtype: int64

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [157]:
df['Age'] = df.groupby('Sex').apply(lambda x: x['Age'].fillna(x['Age'].median())).values
df[['Cabin','Embarked']] = df[['Cabin','Embarked']].fillna('U')

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [159]:
df.loc[(df['PassengerId'] == 6) | (df['PassengerId'] == 20)]['Sex'].isnull()

5     False
19    False
Name: Sex, dtype: bool

Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [160]:
# df.info()

df['Sex'] = df['Sex'].replace(['male', 'female'], [0, 1])
df['Embarked'] = df['Embarked'].replace(['U', 'S', 'C', 'Q'], [0, 1, 2, 3])
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
df.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,26.0,1,0,7.2500,1
1,1,1,1,35.0,1,0,71.2833,2
2,1,3,1,27.0,0,0,7.9250,1
3,1,1,1,55.0,1,0,53.1000,1
4,0,3,0,31.0,0,0,8.0500,1
5,0,3,0,27.0,0,0,8.4583,3
6,0,1,0,15.0,0,0,51.8625,1
7,0,3,0,27.0,3,1,21.0750,1
8,1,3,1,27.0,0,2,11.1333,1
9,1,2,1,14.0,1,0,30.0708,2


### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [161]:
from sklearn.model_selection import train_test_split

In [162]:
y = df['Survived'].values
df.drop(columns=['Survived'], inplace=True)

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [164]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=17)

### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [165]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [180]:
knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(x_train, y_train)

gnb = GaussianNB()
gnb.fit(x_train, y_train)

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)


# Возникает Warning о несходимости при 100 итерациях по умолчанию. Воспользовавшись рецептом 
# https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter,
# я поставил 150 итерации, при 132 уже не сходится.

logreg = LogisticRegression(max_iter=150)
logreg.fit(x_train, y_train)


LogisticRegression(max_iter=150)

Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

Ваш ответ:

1. <br> precision, т.к. лучше пусть будет много ложно-отрицательных срабатываний, чем даже одно ложно-положительное, recall здесь меряет только степень перестраховки
2. <br> recall, т.к. максимально значимы ложно-отрицательные срабатывания, а precision показывает лишь степень перестраховки
3. <br> recall
4. <br> recall
5. <br> precision
6. <br> recall
7. <br> precision

### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.

-- здесь важен recall, потому что в классификаторе мы оцениваем математическую остроту, число попаданий прогноза в количество реально выживших. Если бы мы делали прогноз на будущее, то вероятно важнее был бы precision, т.к. мы искали бы классификатор по возможности совсем без жертв. А здесь важно просто точнее попасть в число, поэтому recall это именно то что надо, он относит верные предсказания о выживших к числу реально выживших.


### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [182]:
from sklearn.metrics import confusion_matrix, recall_score

In [192]:
y_pred_knc = knc.predict(x_test)
m_knc = confusion_matrix(y_test, y_pred_knc)
recall_knc = m_knc[0][0]/(m_knc[0][0] + m_knc[1][0])
recall_knc_ref = recall_score(y_test, y_pred_knc)
print('K Nearest Neighbors classifier')
print(m_knc, f'recall={recall_knc}, reference recall={recall_knc_ref}')
print()

y_pred_gnb = gnb.predict(x_test)
m_gnb = confusion_matrix(y_test, y_pred_gnb)
recall_gnb = m_gnb[0][0]/(m_gnb[0][0] + m_gnb[1][0])
recall_gnb_ref = recall_score(y_test, y_pred_gnb)
print('Naive Bayes Gaussian classifier')
print(m_gnb, f'recall={recall_gnb}, reference recall={recall_gnb_ref}')
print()

y_pred_dtc = dtc.predict(x_test)
m_dtc = confusion_matrix(y_test, y_pred_dtc)
recall_dtc = m_dtc[0][0]/(m_dtc[0][0] + m_dtc[1][0])
recall_dtc_ref = recall_score(y_test, y_pred_dtc)
print('Decision Tree classifier')
print(m_dtc, f'recall={recall_dtc}, reference recall={recall_dtc_ref}')
print()

y_pred_logreg = logreg.predict(x_test)
m_logreg = confusion_matrix(y_test, y_pred_logreg)
recall_logreg = m_logreg[0][0]/(m_logreg[0][0] + m_logreg[1][0])
recall_logreg_ref = recall_score(y_test, y_pred_logreg)
print('Logistic Regression classifier')
print(m_logreg, f'recall={recall_logreg}, reference recall={recall_logreg_ref}')
print()

# Результаты, мягко говоря, удивляют. Вроде всего четыре числа, где тут ошибиться? Проверил, ошибки не нашел.
# Возможно, дело в способе averaging функции recall_score. 

K Nearest Neighbors classifier
[[98 33]
 [50 42]] recall=0.6621621621621622, reference recall=0.45652173913043476

Naive Bayes Gaussian classifier
[[110  21]
 [ 28  64]] recall=0.7971014492753623, reference recall=0.6956521739130435

Decision Tree classifier
[[107  24]
 [ 33  59]] recall=0.7642857142857142, reference recall=0.6413043478260869

Logistic Regression classifier
[[115  16]
 [ 30  62]] recall=0.7931034482758621, reference recall=0.6739130434782609



### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [193]:
from sklearn.metrics import precision_score

In [194]:
### YOUR CODE HERE ###

pres_gnb_ref = precision_score(y_test, y_pred_gnb)

print('Naive Bayes Gaussian classifier')
print(m_gnb, f'reference precision={pres_gnb_ref}')
print()



Naive Bayes Gaussian classifier
[[110  21]
 [ 28  64]] reference precision=0.7529411764705882

